In [10]:
import requests
import xarray as xr 

# List stations

In [11]:
def get_list_stations():

    response = requests.get('https://prod-actris-md.nilu.no/Stations')
    
    stations_demonstrator = []
    
    for station in response.json():

        stations_demonstrator.append({'short_name': station['identifier'],'latitude': station['lat'],'longitude': station['lon'],'long_name': station['name'],'altitude': station['alt']})
    
    return stations_demonstrator

In [12]:
print(get_list_stations())

[{'short_name': 'ABY', 'latitude': 51.858, 'longitude': -4.252, 'long_name': 'Aberystwyth', 'altitude': 15.0}, {'short_name': 'AGI', 'latitude': 35.0381, 'longitude': 33.0578, 'long_name': 'Agia Marina Xyliatou', 'altitude': 520.0}, {'short_name': 'AHT', 'latitude': 62.583333, 'longitude': 24.183333, 'long_name': 'Ahtari', 'altitude': 180.0}, {'short_name': 'AKY', 'latitude': 35.86, 'longitude': 23.31, 'long_name': 'Antikythera', 'altitude': 193.0}, {'short_name': 'ALT', 'latitude': 82.499146, 'longitude': -62.341526, 'long_name': 'Alert', 'altitude': 210.0}, {'short_name': 'AMB', 'latitude': 40.384444, 'longitude': 44.260583, 'long_name': 'Amberd', 'altitude': 2080.0}, {'short_name': 'ANB', 'latitude': 50.571667, 'longitude': 12.998889, 'long_name': 'Annaberg-Buchholz', 'altitude': 545.0}, {'short_name': 'ANM', 'latitude': 44.348, 'longitude': 26.029, 'long_name': 'Bucharest', 'altitude': 93.0}, {'short_name': 'APT', 'latitude': 58.8, 'longitude': 17.383333, 'long_name': 'Aspvreten', 

# List variables

In [13]:
def get_list_variables():

    response = requests.get('https://prod-actris-md.nilu.no/ContentInformation/attributes')
    
    ecv_aerosol = []
    
    for v in response.json():
        
        if 'particle' in v['attribute_type'] or 'aerosol' in v['attribute_type']:
            ecv_aerosol.append(v['attribute_type'])

    ecv_cloud = []

    for v in response.json():

        if 'cloud' in v['attribute_type']:
            ecv_cloud.append(v['attribute_type'])
    
    variables_demonstrator = []

    for var in response.json():

        
        if var['attribute_type'] in ecv_cloud: 
            variables_demonstrator.append({'variable_name': var['attribute_type'], 'ECV_name': ['Cloud Properties']})
        elif var['attribute_type'] in ecv_aerosol:
            variables_demonstrator.append({'variable_name': var['attribute_type'], 'ECV_name': ['Aerosol Properties']})
        else:
            pass
        
    return variables_demonstrator


In [14]:
print(get_list_variables())

[{'variable_name': 'aerosol.absorption.coefficient', 'ECV_name': ['Aerosol Properties']}, {'variable_name': 'aerosol.backscatter.coefficient', 'ECV_name': ['Aerosol Properties']}, {'variable_name': 'aerosol.backscatter.coefficient.hemispheric', 'ECV_name': ['Aerosol Properties']}, {'variable_name': 'aerosol.backscatter.ratio', 'ECV_name': ['Aerosol Properties']}, {'variable_name': 'aerosol.depolarisation.coefficient', 'ECV_name': ['Aerosol Properties']}, {'variable_name': 'aerosol.depolarisation.ratio', 'ECV_name': ['Aerosol Properties']}, {'variable_name': 'aerosol.extinction.coefficient', 'ECV_name': ['Aerosol Properties']}, {'variable_name': 'aerosol.extinction.ratio', 'ECV_name': ['Aerosol Properties']}, {'variable_name': 'aerosol.extinction.to.backscatter.ratio', 'ECV_name': ['Aerosol Properties']}, {'variable_name': 'aerosol.optical.depth', 'ECV_name': ['Aerosol Properties']}, {'variable_name': 'aerosol.optical.depth.550', 'ECV_name': ['Aerosol Properties']}, {'variable_name': 'a

# List datasets

In [15]:
def list_datasets(variables,temporal_extent,spatial_extent):
    
    temporal_extent = [start_time, end_time]
    spatial_extent=[lon0, lat0, lon1, lat1]

    dataset_endpoints = []
    
    headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
    }

    data = '{"where":{"argument":{"type":"content","sub-type":"attribute_type","value":'+ str(variables) +',"case-sensitive":false,"and":{"argument":{"type":"temporal_extent","comparison-operator":"overlap","value":["'+ temporal_extent[0] +'","'+ temporal_extent[1] +'"]}}}}}'

    response = requests.post('https://prod-actris-md.nilu.no/Metadata/query', headers=headers, data=data)

    for ds in response.json():

        
        # spatial extent
    

        lat_point, lon_point = ds['md_data_identification']['station']['lat'], ds['md_data_identification']['station']['lon']

        if (lon0 < lon_point < lon1) and (lat0 < lat_point < lat1) == True:
            dataset_endpoints.append(ds['md_distribution_information']['dataset_url'])
        else:
            pass
            
    return dataset_endpoints

In [16]:
# bounding box around Norway
lon0, lat0, lon1, lat1 = 1.99207807783, 42.0788841824, 41.29341841, 80.6571442736
list_of_variables = get_list_variables()
variables = [var['variable_name'] for var in list_of_variables]
start_time = "1970-01-01"
end_time = "2020-01-01"

temporal_extent=[start_time,end_time]
spatial_extent=[lon0, lat0, lon1, lat1]

# example only fetching the first variable
#datasets = list_datasets(variables[0:1],temporal_extent,spatial_extent)
datasets = list_datasets(variables,temporal_extent,spatial_extent)

print(datasets[0:100])

['http://thredds.nilu.no/thredds/dodsC/ebas/NO0042G.20000301000000.20170523103005.dmps.particle_number_size_distribution.aerosol.6y.1h.NO01L_dmps_no42.NO01L_dmps.lev2.nc', 'https://thredds.nilu.no/thredds/dodsC/ebas/SE0023R.20170101000000.20210122094244.filter_absorption_photometer.aerosol_absorption_coefficient.pm10.3y.1h.SE03L_Thermo-MAAP_5012-368.SE03L_abs_coef_MAAP_v1.lev2.nc', 'https://thredds.nilu.no/thredds/dodsC/ebas/SE0023R.20170101000000.20201109204520.dmps.particle_number_size_distribution.pm10.3y.1h.SE03L_MILOU.SE03L_DMPS_EUSAAR.lev2.nc', 'http://thredds.nilu.no/thredds/dodsC/ebas/NO0002R.20151217090000.20170504082503.dmps.particle_number_size_distribution.pm10.72w.1h.NO01L_NILU_DMPSmodel2_BIR_NRT.NO01L_dmps_DMPS_BIR05..nc', 'https://thredds.nilu.no/thredds/dodsC/ebas/NL0011R.20140101000000.20180703102718.filter_absorption_photometer.aerosol_absorption_coefficient.pm10.2y.1h.NL02L_Thermo_5012_TNO.NL02L_abs_coef.lev1.nc', 'https://thredds.nilu.no/thredds/dodsC/ebas/NL0011R.2

# Read datasets

In [17]:
def read_dataset(url, variables):

    # map variables
    mapping = {'aerosol.absorption.coefficient':'Aerosol Properties',
                'aerosol.backscatter.coefficient':'Aerosol Properties',
                'aerosol.backscatter.coefficient.hemispheric':'Aerosol Properties',
                'aerosol.backscatter.ratio':'Aerosol Properties',
                'aerosol.depolarisation.coefficient':'Aerosol Properties',
                'aerosol.depolarisation.ratio':'Aerosol Properties',
                'aerosol.extinction.coefficient':'Aerosol Properties',
                'aerosol.extinction.ratio':'Aerosol Properties',
                'aerosol.extinction.to.backscatter.ratio':'Aerosol Properties',
                'aerosol.optical.depth':'Aerosol Properties',
                'aerosol.optical.depth.550':'Aerosol Properties',
                'aerosol.rayleigh.backscatter':'Aerosol Properties',
                'aerosol.scattering.coefficient':'Aerosol Properties',
                'cloud.aerosol.target.classification':'Cloud Properties',
                'cloud.fraction':'Cloud Properties',
                'cloud.mask':'Cloud Properties',
                'particle.number.concentration':'Aerosol Properties',
                'particle.number.size.distribution':'Aerosol Properties'}
    
    # For InSitu specific variables
    actris_mapping = {'particle_number_size_distribution':'particle.number.size.distribution',
                    'aerosol_absorption_coefficient':'aerosol.absorption.coefficient',
                    'aerosol_light_backscattering_coefficient':'aerosol.backscatter.coefficient.hemispheric',
                    'aerosol_light_backscattering_coefficient':'aerosol.backscatter.coefficient',
                    'aerosol_light_scattering_coefficient':'aerosol.scattering.coefficient',
                    None:'aerosol.backscatter.ratio',
                    None:'aerosol.depolarisation.coefficient',
                    None:'aerosol.depolarisation.ratio',
                    None:'aerosol.extinction.coefficient',
                    None:'aerosol.extinction.ratio',
                    None:'aerosol.extinction.to.backscatter.ratio',
                    None:'aerosol.optical.depth',
                    None:'aerosol.optical.depth.550',
                    None:'aerosol.rayleigh.backscatter',
                    None:'cloud.aerosol.target.classification',
                    None:'cloud.fraction',
                    None:'cloud.mask',
                    'particle_number_concentration':'particle.number.concentration'}

    varlist_tmp = []
    for k,v in mapping.items():
        if v in variables:
            varlist_tmp.append(k)

    actris_varlist = []
    for k,v in actris_mapping.items():
        if v in varlist_tmp:
            actris_varlist.append(k)

    print(actris_varlist)

    ds = xr.open_dataset(url)
    var_list = []
    for varname, da in ds.data_vars.items():
        if 'metadata' in varname or 'time' in varname or '_qc' in varname:
            pass
        #ext in url_string for ext in extensionsToCheck
        elif any(var in varname for var in actris_varlist):
            var_list.append(varname)
        else:
            pass
    print("varlist: ", var_list)
    return ds[var_list]

In [18]:
# InSitu
variables = ['Aerosol Properties']
url = 'http://thredds.nilu.no/thredds/dodsC/ebas/DE0044R.20200101000000.20200428062924.filter_absorption_photometer.aerosol_absorption_coefficient.pm10.17w.1h.DE08L_Thermo_5012_MEL.DE08L_MAAP_MEL.lev2.nc'
ds = read_dataset(url, variables)
print(ds)

['particle_number_size_distribution', 'aerosol_absorption_coefficient', 'aerosol_light_backscattering_coefficient', 'aerosol_light_scattering_coefficient', 'particle_number_concentration']
varlist:  ['aerosol_absorption_coefficient_prec1587', 'aerosol_absorption_coefficient_amean', 'aerosol_absorption_coefficient_perc8413']
<xarray.Dataset>
Dimensions:                                  (Wavelength: 1, time: 2446)
Coordinates:
  * Wavelength                               (Wavelength) float64 670.0
  * time                                     (time) datetime64[ns] 2020-01-01...
Data variables:
    aerosol_absorption_coefficient_prec1587  (Wavelength, time) float64 ...
    aerosol_absorption_coefficient_amean     (Wavelength, time) float64 ...
    aerosol_absorption_coefficient_perc8413  (Wavelength, time) float64 ...
Attributes:
    Conventions:                   CF-1.7, ACDD-1.3
    featureType:                   timeSeries
    title:                         Ground based in situ observat